In [1]:
pip install pyyaml

In [4]:
import os
import yaml
import pandas as pd
from collections import defaultdict

# Define input and output directories
input_dir = r"C:\Users\usen\Mini project 2\data"
output_dir = os.path.join(input_dir, "output_csv")

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Dictionary to store data by ticker
symbol_data = defaultdict(list)

# Recursively search for YAML files in subdirectories
for root, _, files in os.walk(input_dir):
    for file_name in files:
        if file_name.endswith(".yaml") or file_name.endswith(".yml"):
            file_path = os.path.join(root, file_name)
            
            with open(file_path, "r") as file:
                data = yaml.safe_load(file)
            
            # Process each stock entry in YAML file
            for entry in data:
                ticker = entry["Ticker"]
                
                # Append data to the corresponding ticker's list
                symbol_data[ticker].append(entry)

# Save each ticker's data into its own folder
for ticker, records in symbol_data.items():
    ticker_folder = os.path.join(output_dir, ticker)
    os.makedirs(ticker_folder, exist_ok=True)
    
    csv_path = os.path.join(ticker_folder, f"{ticker}.csv")
    df = pd.DataFrame(records)
    df.to_csv(csv_path, index=False)

print("Data extraction and transformation completed successfully!")


Data extraction and transformation completed successfully!


In [5]:
import os
import pandas as pd
import streamlit as st
import matplotlib.pyplot as plt

# Define data paths
data_dir = r"C:\Users\usen\Mini project 2\data\output_csv"  # Folder containing ticker-wise CSVs
sector_mapping_file = r"C:\Users\usen\Mini project 2\data\sector\cleaned_sector_mapping.csv"  # Sector mapping CSV

# Load sector mapping
sector_mapping = pd.read_csv(sector_mapping_file)

# Ensure required columns exist
if "Ticker" not in sector_mapping.columns or "sector" not in sector_mapping.columns:
    st.error("Sector mapping CSV must contain 'Ticker' and 'sector' columns!")
    st.stop()

# Initialize dataframe to store yearly returns
sector_returns = []

# Process each stock CSV file
for ticker in os.listdir(data_dir):
    ticker_folder = os.path.join(data_dir, ticker)
    csv_path = os.path.join(ticker_folder, f"{ticker}.csv")
    
    if os.path.isfile(csv_path):
        df = pd.read_csv(csv_path)

        # Ensure required columns exist
        if "date" not in df.columns or "close" not in df.columns:
            st.warning(f"Skipping {ticker}: Missing 'date' or 'close' column")
            continue
        
        # Convert date column to datetime format
        df["date"] = pd.to_datetime(df["date"])
        df["year"] = df["date"].dt.year
        
        # Calculate yearly return (percentage change)
        yearly_returns = df.groupby("year")["close"].pct_change().mean() * 100  # Convert to percentage
        
        # Get sector for the ticker
        sector_row = sector_mapping[sector_mapping["Ticker"] == ticker]
        if not sector_row.empty:
            sector = sector_row["sector"].values[0]
            sector_returns.append({"Sector": sector, "Yearly Return": yearly_returns.mean()})

# Convert to DataFrame
sector_df = pd.DataFrame(sector_returns)

# Aggregate average returns per sector
sector_avg_returns = sector_df.groupby("Sector")["Yearly Return"].mean().reset_index()

# Streamlit UI
st.title("📊 Sector-wise Performance Analysis")
st.write("This chart shows the **average yearly return** for each sector.")

# Plot bar chart
fig, ax = plt.subplots(figsize=(10, 5))
ax.bar(sector_avg_returns["Sector"], sector_avg_returns["Yearly Return"], color="skyblue")
ax.set_xlabel("Sector")
ax.set_ylabel("Average Yearly Return (%)")
ax.set_title("Sector-wise Stock Performance")
plt.xticks(rotation=45)
st.pyplot(fig)


2025-03-01 18:16:37.745 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 18:16:39.162 
  command:

    streamlit run C:\Users\usen\AppData\Local\Programs\Python\Python313\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-03-01 18:16:39.165 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 18:16:39.170 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 18:16:39.178 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 18:16:39.187 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 18:16:39.200 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 18:16:3

DeltaGenerator()